# 📚 Goodreads Data Cleaning & EDA 

This project transforms raw scraped Goodreads metadata into clean, structured data ready for analysis.

---

## 🧼 Data Cleaning & Transformation Workflow

- Assigned a unique `book_id` to each book for consistent identification.  
- Normalized the multi-valued `genres` column into a separate dataframe `book_genres_df`.  
- Cleaned textual fields (`title`, `author`, `description`) by removing excess whitespace.  
- Imputed missing `language` values using `langid` based on book descriptions or titles.  
- Converted `num_pages` and `publication_year` to integer types, removing invalid entries.
- Detected and removed duplicate records based on `title` and `author` combinations to ensure data uniqueness.  

---

## 📊 Exploratory Data Analysis (EDA)

- Created a custom `print_summary()` function to generate detailed diagnostics for data quality and distribution.  
- Summarized categorical variables with counts, missing percentages, unique values, and entropy-based diversity.  
- Summarized numeric variables with key statistics including min, max, mean, median, and standard deviation.  

---


## 🛠️ Tools & Libraries

- **Python**: Core data cleaning, transformation, and orchestration logic  
- **Pandas / SciPy**: Data wrangling, numerical analysis, and entropy-based diagnostics  
- **langid**: Automated language detection for imputation of missing values  
- **Jupyter Notebook**: Interactive development, function documentation, and reproducible EDA

---

## 📤 Output

- `../data/goodreads-books-clean.csv`: Cleaned metadata for each book, including normalized columns, imputed values, and engineered features like `book_age` and `author_popularity`.
- `../data/goodreads-book-genres-clean.csv`: One-hot encoded genre matrix, aligned with `book_id` from the cleaned books dataset.
- All files are **fully refreshed on each run**, ensuring up-to-date, reproducible data for downstream analysis.

---

## ✅ Outcomes

- Built a reusable, production-style data preparation pipeline.
- Prepared clean, analysis-ready datasets suitable for BI tools like **Power BI** or future machine learning applications.
---

## Importing Libraries & Notebook Setup

In [1]:
# Data processing
import pandas as pd 
from scipy.stats import entropy
import ast
import langid

# Notebook display enhancements
from IPython.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

## Load Raw Dataset

In [2]:
# Read CSV file with data and print the first couple rows
books_df = pd.read_csv('../data/goodreads-books-raw.csv')
books_df.head()

,title,author,description,genres,language,num_pages,publication_year,rating,num_ratings,num_reviews,part_of_series,series_name,cover_link,book_link,series_link
0,The Hunger Games,Suzanne Collins,Winning means fame and fortune. Losing means c...,"['Young Adult', 'Dystopia', 'Fiction', 'Fantas...",English,374,2008,4.35,9541172,249053,True,The Hunger Games,https://images-na.ssl-images-amazon.com/images...,https://www.goodreads.com/book/show/2767052-th...,https://www.goodreads.com/series/73758-the-hun...
1,Pride and Prejudice,Jane Austen,"Since its immediate success in 1813, Pride an...","['Classics', 'Romance', 'Fiction', 'Historical...",English,279,1813,4.29,4605881,135792,False,NaN,https://images-na.ssl-images-amazon.com/images...,https://www.goodreads.com/book/show/1885.Pride...,NaN
2,To Kill a Mockingbird,Harper Lee,"""Shoot all the bluejays you want, if you can h...","['Classics', 'Fiction', 'Historical Fiction', ...",English,323,1960,4.26,6658977,127709,True,To Kill a Mockingbird,https://images-na.ssl-images-amazon.com/images...,https://www.goodreads.com/book/show/2657.To_Ki...,https://www.goodreads.com/series/255474-to-kil...
3,Harry Potter and the Order of the Phoenix,J.K. Rowling,It's official: the evil Lord Voldemort has ret...,"['Fantasy', 'Young Adult', 'Fiction', 'Magic',...",English,896,2003,4.50,3674686,73546,True,Harry Potter,https://images-na.ssl-images-amazon.com/images...,https://www.goodreads.com/book/show/58613451-h...,https://www.goodreads.com/series/45175-harry-p...
4,The Book Thief,Markus Zusak,It is 1939. Nazi Germany. The country is holdi...,"['Historical Fiction', 'Fiction', 'Young Adult...",English,592,2005,4.39,2790201,155737,False,NaN,https://images-na.ssl-images-amazon.com/images...,https://www.goodreads.com/book/show/19063.The_...,NaN


## Normalize Genres into Separate Table

In [3]:
# Add an ID column (1-based index)
books_df.insert(0, 'book_id', books_df.index + 1)

# Extract 'book_id' and 'genres' columns into a separate DataFrame
book_genres_df = books_df[['book_id', 'genres']].copy()

# Drop 'genres' column from the original DataFrame
books_df.drop(columns = 'genres', inplace = True)

### 📊 Function: `print_summary`

Generates a clean, readable diagnostic summary of a Pandas DataFrame to assist in Exploratory Data Analysis (EDA). This function helps identify missing values, understand categorical diversity, and observe key statistics for numeric columns. 

In [4]:
def print_summary(
    df: pd.DataFrame,
) -> pd.DataFrame | None:
    """
    Summarizes DataFrame structure and key statistics.

    Parameters:
        df (pd.DataFrame): Data to summarize.

    Returns:
        None.
    """
    
    print(f"Overall Shape of DataFrame: {df.shape}")
    print()
    print(f"Categorical Measures")
    print(f"{'Column Name':<25} {'Count':<15} {'% Missing': <15} {'Unique':<10} {'Diversity':<20}")
    print(f"{'-'*90}")
    
    categorical_cols = df.select_dtypes(include = ['object', 'category']).columns.tolist()
    
    for col in categorical_cols:
        count = df[col].notnull().sum()
        pct_missing = round(df[col].isna().sum() / len(df) * 100, 1)  # Percent missing over total rows
        unique_values = df[col].nunique()

        value_counts = df[col].value_counts(normalize = True, dropna = True)
        diversity = round(entropy(value_counts, base = 2), 2)

        print(f"{col:<25} {count:<15} {pct_missing:>6.2f}%        {unique_values:<10} {diversity:<20}")
        
    
    print()
    print(f"Numeric Measures")
    print(f"{'Column Name':<25} {'Min Value':<15} {'Max Value':<15} {'Mean Value':<15} {'Median Value':<15} {'Std Dev':<20}")
    print(f"{'-'*90}")
    
    numeric_cols = df.select_dtypes(include = ['number']).columns.tolist()
    
    for col in numeric_cols:
        min_value = df[col].min()
        max_value = df[col].max()
        mean_value = df[col].mean()
        median_value = df[col].median()
        std_dev = df[col].std()

        print(f"{col:<25} {min_value:<15} {max_value:<15} {mean_value:<15.2f} {median_value:<15.2f} {std_dev:<20.2f}")
            
            
print_summary(books_df)

Overall Shape of DataFrame: (9991, 15)

Categorical Measures
Column Name               Count           % Missing       Unique     Diversity           
------------------------------------------------------------------------------------------
title                     9991              0.00%        9617       13.2                
author                    9991              0.00%        4519       11.42               
description               9984              0.10%        9929       13.27               
language                  9925              0.70%        30         0.31                
num_pages                 9991              0.00%        1024       8.97                
publication_year          9991              0.00%        322        6.23                
series_name               5335             46.60%        2647       10.86               
cover_link                9991              0.00%        9984       13.28               
book_link                 9991              0.

### 📝 Function: `sanitize_integer_columns`

Cleans specified DataFrame columns by converting them to integer type, with errors and missing values removed during conversion. This helps ensure numeric columns are in a 
consistent format suitable for analysis.

In [5]:
def sanitize_integer_columns(
    df: pd.DataFrame, 
    col_names: list[str]
) -> pd.DataFrame:
    """
    Converts given columns to integer type, removing rows with non-convertible values.

    Steps:
    - Attempts to coerce each column’s values to numeric (allowing NaNs)
    - Drops rows where conversion failed (NaNs introduced)
    - Casts the cleaned column to integer dtype

    Parameters:
        df (pd.DataFrame): The DataFrame to clean.
        col_names (list[str]): List of column names to convert.

    Returns:
        pd.DataFrame: Updated DataFrame with specified columns as integers and invalid rows removed.
    """
    for col in col_names:
        df[col] = pd.to_numeric(df[col], errors = 'coerce')
        df = df.dropna(subset = [col])
        df.loc[:, col] = df[col].astype(int)
    return df

# Application
books_df = sanitize_integer_columns(books_df, ["num_pages", "publication_year"])

C:\Users\Chris\AppData\Local\Temp\ipykernel_57404\2627322109.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = pd.to_numeric(df[col], errors = 'coerce')


### 🌐 Function: `impute_missing_languages`

Fills in missing entries in the `language` column by intelligently classifying the language of each book based on its description or title using the `langid` library.

In [6]:
def impute_missing_languages(df: pd.DataFrame) -> pd.DataFrame:
    """
    Fills missing values in the 'language' column using langid classification.
    Prioritizes classifying from 'description', and falls back to 'title' if description is missing.

    Parameters:
        df (pd.DataFrame): Input DataFrame with a 'language' column.

    Returns:
        pd.DataFrame: Updated DataFrame with missing languages filled.
    """

    # Create a copy to avoid modifying the original DataFrame directly
    df = df.copy()

    # Define a helper function to classify language per row
    def classify_language(row):
        if pd.isna(row['language']):
            text = row['description']
            if not isinstance(text, str) or text.strip() == "":
                text = row['title']
            return langid.classify(text)[0]
        return row['language']

    # Apply the language classifier to the DataFrame
    df['language'] = df.apply(classify_language, axis = 1)

    return df

# Application
books_df = impute_missing_languages(books_df)

### 🧹 Function: `remove_duplicate_books`

Removes duplicate rows from a DataFrame based on specified columns, typically used to ensure unique book entries by title and author.

In [7]:
def remove_duplicate_books(
    df: pd.DataFrame,
    subset_cols: list = ["title", "author"],
    keep: str = "first"
) -> pd.DataFrame:
    """
    Remove duplicate rows based on specified subset of columns.

    Parameters:
        df (pd.DataFrame): Input DataFrame.
        subset_cols (list): Columns to consider for identifying duplicates. Default is ['title', 'author'].
        keep (str): Which duplicate to keep. Options are 'first', 'last', or False. Default is 'first'.

    Returns:
        pd.DataFrame: DataFrame with duplicates removed.
    """
    return df.drop_duplicates(subset = subset_cols, keep = keep)

# Application
books_df = remove_duplicate_books(books_df)

### 🧹 Function: `clean_whitespace`

Cleans leading, trailing, and excessive intermediate whitespace from specified DataFrame columns.  

In [8]:
def clean_whitespace(
    df: pd.DataFrame,
    subset_cols: list = ["title", "author", "description"]
) -> pd.DataFrame:
    """
    Remove leading, trailing, and excessive intermediate whitespace from specified columns.
    
    Parameters:
        df (pd.DataFrame): Input DataFrame.
        subset_cols (list): Columns to consider for whitespace cleaning. Default is ["title", "author", "description"].
        
    Returns:
        pd.DataFrame: DataFrame with whitespace cleaned in specified columns.
    """
    
    def _clean_text(text):
        if not isinstance(text, str):
            return text  # Return as is if not a string (handle NaNs or other types)
        return ' '.join(text.split())
    
    for col in subset_cols:
        if col in df.columns:
            df[col] = df[col].apply(_clean_text)
    
    return df

# Application
books_df = clean_whitespace(books_df)

In [9]:
print_summary(books_df)

Overall Shape of DataFrame: (9715, 15)

Categorical Measures
Column Name               Count           % Missing       Unique     Diversity           
------------------------------------------------------------------------------------------
title                     9715              0.00%        9532       13.21               
author                    9715              0.00%        4484       11.43               
description               9709              0.10%        9702       13.24               
language                  9715              0.00%        31         0.36                
series_name               5175             46.70%        2636       10.88               
cover_link                9715              0.00%        9714       13.25               
book_link                 9715              0.00%        9715       13.25               
series_link               5175             46.70%        2655       10.89               

Numeric Measures
Column Name               Mi

## Feature Engineering

In [10]:
from datetime import datetime

current_year = datetime.now().year

books_df['book_age'] = current_year - books_df['publication_year']

In [11]:
author_counts = books_df['author'].value_counts().to_dict()
books_df['author_popularity'] = books_df['author'].map(author_counts)

### Explore book_genres_df

In [12]:
book_genres_df.head()

,book_id,genres
0,1,"['Young Adult', 'Dystopia', 'Fiction', 'Fantas..."
1,2,"['Classics', 'Romance', 'Fiction', 'Historical..."
2,3,"['Classics', 'Fiction', 'Historical Fiction', ..."
3,4,"['Fantasy', 'Young Adult', 'Fiction', 'Magic',..."
4,5,"['Historical Fiction', 'Fiction', 'Young Adult..."


In [13]:
def encode_genres(
    df_genres: pd.DataFrame,
    df_books: pd.DataFrame,
    genre_col: str = 'genres',
    book_id_col: str = 'book_id'
) -> pd.DataFrame:
    """
    One-hot encodes the genres in the specified DataFrame column and filters 
    to only include books present in the main books DataFrame.

    Parameters:
        df_genres (pd.DataFrame): DataFrame containing genre data with a book ID column.
        df_books (pd.DataFrame): Reference DataFrame containing valid book IDs.
        genre_col (str): Name of the column containing genre lists. Default is 'genres'.
        book_id_col (str): Name of the book ID column to use for alignment. Default is 'book_id'.

    Returns:
        pd.DataFrame: One-hot encoded genre DataFrame aligned to df_books.
    """

    # Filter to only include books present in df_books
    df_filtered = df_genres[df_genres[book_id_col].isin(df_books[book_id_col])].copy()

    # Convert genre strings to actual lists (if stored as strings)
    df_filtered[genre_col] = df_filtered[genre_col].apply(ast.literal_eval)

    # Get all unique genres
    unique_genres = sorted({genre for genre_list in df_filtered[genre_col] for genre in genre_list})

    # Define one-hot encoding function
    def one_hot_encode(genre_list):
        return [1 if genre in genre_list else 0 for genre in unique_genres]

    # Apply encoding and join back to book IDs
    genre_dummies = pd.DataFrame(
        df_filtered[genre_col].apply(one_hot_encode).tolist(),
        columns=unique_genres
    )

    # Combine with original DataFrame (minus genre list)
    df_encoded = pd.concat(
        [df_filtered[[book_id_col]].reset_index(drop=True), genre_dummies.reset_index(drop=True)],
        axis=1
    )

    return df_encoded

book_genres_df = encode_genres(book_genres_df, books_df)

In [14]:
# Check if the two DataFrames have the same number of rows
print(f"books_df: {len(books_df)}, book_genres_df: {len(book_genres_df)}")
print("Lengths match:" , len(books_df) == len(book_genres_df))

books_df: 9715, book_genres_df: 9715
Lengths match: True


In [15]:
book_genres_df.head()

,book_id,12th Century,13th Century,15th Century,16th Century,17th Century,18th Century,19th Century,20th Century,21st Century,...,World History,World War I,World War II,Writing,Young Adult,Young Adult Contemporary,Young Adult Fantasy,Young Adult Paranormal,Young Adult Romance,Zombies
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
1,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
4,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0


## Migrate Data to CSV Files

In [16]:
books_df.to_csv('../data/goodreads-books-clean.csv', index = False)

book_genres_df.to_csv('../data/goodreads-book-genres-clean.csv', index = False)